In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pyodbc
import mysql.connector
import json
import matplotlib.pyplot as plt
import seaborn as sns
from decimal import Decimal

import requests
import datetime
from tqdm import tqdm

pd.set_option('display.max_columns', None)


In [2]:
pyodbc.drivers()

['MySQL']

In [3]:
# Redshift 연결 정보
host=''
dbname=''
user=''
password=''
port=''

In [4]:
# Redshift에 연결
conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password,
    port=port
)

In [5]:
# 커서 생성
cursor = conn.cursor()
# SQL 쿼리 실행 (예시: frigga_sample 테이블에서 모든 데이터를 조회)
cursor.execute("""
                SELECT
                    *
                FROM "data_analysis_db"."da_schema"."t_frigga_weather_map_alert"
                ;
               """)

# 결과 가져오기
rows = cursor.fetchall()

cursor.close()
conn.close()

In [6]:
# 컬럼명 리스트 정의
columns = ['device_datetime_kor', 'device_serial', 'lat', 'lon', 'battery', 'temperature', 'humidity', 'acc_orig', 'acc'
           , 'accx', 'accy', 'accz', 'tiltx', 'tilty', 'tiltz', 'measure_data_kst', 'aws_station_id', 'aws_station_name'
           , 'aws_lat', 'aws_lon', 'distance_km', 'ws1', 'ws10', 'ta', 'hm', 'min_temp', 'max_temp', 'mean_temp', 'std_temp'
           , 'min_humi', 'max_humi', 'mean_humi', 'std_humi'
           , 'min_acc', 'max_acc', 'mean_acc', 'std_acc'
           , 'q1_temp', 'q3_temp', 'temp_start', 'temp_end'
           , 'q1_humi', 'q3_humi', 'humi_start', 'humi_end'
           , 'q1_acc', 'q3_acc', 'acc_start', 'acc_end'
           , 'alert_temp', 'alert_humi', 'alert_acc', 'item_shock'
           ]

# rows를 DataFrame으로 변환
df = pd.DataFrame(rows, columns=columns)

# 데이터프레임 확인
print("데이터프레임 크기:", df.shape)
print("\n처음 5행 데이터:")
print(df.head())

데이터프레임 크기: (146528, 53)

처음 5행 데이터:
  device_datetime_kor device_serial        lat         lon battery  \
0 2024-11-26 12:31:32      VC6M6034  36.218604  127.883799    80.0   
1 2024-11-28 19:12:18      VC6M6094  34.796662  128.045073    50.0   
2 2024-11-28 22:12:18      VC6M6094  34.633926  127.978666    50.0   
3 2024-11-28 04:14:10      VC6M60A2  34.634202  127.985764    40.0   
4 2024-12-17 21:31:47      VC6M60AD  36.815106  127.061828   100.0   

  temperature humidity acc_orig   acc  accx  accy accz tiltx  tilty  tiltz  \
0        11.8     60.0      1.5  1.51  -0.1  -1.5  0.1  -2.0  -85.4    4.2   
1        10.8     56.0      1.1  1.10   0.0  -1.1  0.0  -0.2  -88.4    1.6   
2        10.2     58.0      1.1  1.10   0.0  -1.1  0.0  -0.6  -88.4    1.5   
3         8.1     56.0      1.1  1.10   0.0  -1.1  0.0   1.7  -87.0    2.5   
4         3.2     55.0      1.0  1.01   0.1   0.1  1.0   2.5    3.5  -85.7   

     measure_data_kst  aws_station_id aws_station_name      aws_lat  \
0 2

In [7]:
df.to_csv('/Users/stella/Documents/03.proj/01.modeling/01.prediction/data/t_frigga_weather_map_alert_250214.csv', index=False, encoding='euc-kr')
df.head(2)

,device_datetime_kor,device_serial,lat,lon,battery,temperature,humidity,acc_orig,acc,accx,accy,accz,tiltx,tilty,tiltz,measure_data_kst,aws_station_id,aws_station_name,aws_lat,aws_lon,distance_km,ws1,ws10,ta,hm,min_temp,max_temp,mean_temp,std_temp,min_humi,max_humi,mean_humi,std_humi,min_acc,max_acc,mean_acc,std_acc,q1_temp,q3_temp,temp_start,temp_end,q1_humi,q3_humi,humi_start,humi_end,q1_acc,q3_acc,acc_start,acc_end,alert_temp,alert_humi,alert_acc,item_shock
0,2024-11-26 12:31:32,VC6M6034,36.218604,127.883799,80.0,11.8,60.0,1.5,1.51,-0.1,-1.5,0.1,-2.0,-85.4,4.2,2024-11-26 12:00:00,135,추풍령,36.22025000,127.99458000,9.962353,5.20,5.40,8.50,75.60,-5.50,21.30,12.81,51.36,32.00,87.00,53.29,8.78,0.88,1.81,1.16,0.10,7.50,20.10,-11.40,39.00,46.00,60.00,25.00,81.00,1.06,1.21,0.84,1.43,정상,정상,일탈,파손
1,2024-11-28 19:12:18,VC6M6094,34.796662,128.045073,50.0,10.8,56.0,1.1,1.10,0.0,-1.1,0.0,-0.2,-88.4,1.6,2024-11-28 19:00:00,913,상주면,34.72380000,127.98155000,9.957861,5.00,5.20,8.00,54.50,8.00,17.40,11.55,3.97,49.00,65.00,60.04,2.64,1.10,1.50,1.14,0.08,10.15,12.50,6.62,16.03,59.00,62.00,54.50,66.50,1.10,1.10,1.10,1.10,정상,정상,정상,정상


In [8]:
# 일탈 상태를 이진 변수로 변환 (예: '정상'은 0, '일탈'은 1)
df['alert_temp'] = df['alert_temp'].apply(lambda x: 1 if x == '일탈' else 0)
df['alert_humi'] = df['alert_humi'].apply(lambda x: 1 if x == '일탈' else 0)
df['alert_acc'] = df['alert_acc'].apply(lambda x: 1 if x == '일탈' else 0)

In [ ]:
# median ws1, ws10, ta, hm의 평균값 계산
median_values = df.groupby('device_serial')[['ws1', 'ws10', 'ta', 'hm']].median().reset_index()
median_values.rename(columns={'ws1': 'median_ws1', 'ws10': 'median_ws10', 'ta': 'median_ta', 'hm': 'median_hm'}, inplace=True)
# df.rename(columns={'ws1': 'median_ws1', 'ws10': 'median_ws10', 'ta': 'median_ta', 'hm': 'median_hm'}, inplace=True)
# 조건에 맞는 값 대체
df['ws1'] = df['ws1'].where(df['ws1'] > -50, df['median_ws1'])
df['ws10'] = df['ws10'].where(df['ws10'] > -50, df['median_ws10'])
df['ta'] = df['ta'].where(df['ta'] > -50, df['median_ta'])
df['hm'] = df['hm'].where(df['hm'] > -50, df['median_hm'])

In [10]:
median_values[median_values['device_serial']=='VC6M6068']

,device_serial,median_ws1,median_ws10,median_ta,median_hm
66,VC6M6068,1.4,1.5,7.2,87.7


In [20]:
df[df['device_serial']=='VC6M6068']

,device_datetime_kor,device_serial,lat,lon,battery,temperature,humidity,acc_orig,acc,accx,accy,accz,tiltx,tilty,tiltz,measure_data_kst,aws_station_id,aws_station_name,aws_lat,aws_lon,distance_km,ws1,ws10,ta,hm,min_temp,max_temp,mean_temp,std_temp,min_humi,max_humi,mean_humi,std_humi,min_acc,max_acc,mean_acc,std_acc,q1_temp,q3_temp,temp_start,temp_end,q1_humi,q3_humi,humi_start,humi_end,q1_acc,q3_acc,acc_start,acc_end,alert_temp,alert_humi,alert_acc,item_shock
104117,2024-11-21 01:41:12,VC6M6068,36.808531,127.056090,100.0,7.9,75.0,1.1,1.1,-0.1,-1.1,0.0,-7.2,-82.5,2.1,2024-11-21 01:00:00,493,송악,36.73615000,127.00805000,9.105699,2.20,1.70,8.00,88.20,6.40,8.30,7.36,0.42,69.00,75.00,72.79,1.23,1.00,1.40,1.05,0.11,6.80,7.98,5.04,9.74,72.00,73.00,70.50,74.50,1.00,1.00,1.00,1.00,0,0,1,정상
0,2024-11-21 01:51:12,VC6M6068,36.853514,127.043575,100.0,8.0,75.0,1.4,1.4,0.0,-1.4,0.0,0.0,-87.4,2.6,2024-11-21 01:00:00,617,직산,36.89056000,127.14556000,9.979244,0.90,0.80,5.70,91.70,6.40,8.30,7.36,0.42,69.00,75.00,72.79,1.23,1.00,1.40,1.05,0.11,6.80,7.98,5.04,9.74,72.00,73.00,70.50,74.50,1.00,1.00,1.00,1.00,0,0,1,정상
117413,2024-11-21 02:01:12,VC6M6068,36.960435,126.985483,100.0,8.0,75.0,1.3,1.3,0.0,-1.3,0.0,-0.6,-87.5,2.4,2024-11-21 02:00:00,358,현덕면,36.96720000,126.92140000,5.756006,-99.50,-99.90,6.20,84.90,6.40,8.30,7.36,0.42,69.00,75.00,72.79,1.23,1.00,1.40,1.05,0.11,6.80,7.98,5.04,9.74,72.00,73.00,70.50,74.50,1.00,1.00,1.00,1.00,0,0,1,정상
138865,2024-11-21 02:11:12,VC6M6068,37.032959,126.982620,100.0,8.1,74.0,1.4,1.4,0.0,-1.4,-0.1,0.4,-86.8,3.2,2024-11-21 02:00:00,374,청북,37.04670000,126.97060000,1.862537,0.00,0.00,6.00,90.10,6.40,8.30,7.36,0.42,69.00,75.00,72.79,1.23,1.00,1.40,1.05,0.11,6.80,7.98,5.04,9.74,72.00,73.00,70.50,74.50,1.00,1.00,1.00,1.00,0,0,1,정상
111609,2024-11-21 02:21:12,VC6M6068,37.081548,126.857601,100.0,8.3,73.0,1.1,1.1,0.0,-1.1,0.0,0.3,-85.4,4.6,2024-11-21 02:00:00,515,운평,37.08453000,126.77389000,7.450497,1.40,1.60,7.30,84.70,6.40,8.30,7.36,0.42,69.00,75.00,72.79,1.23,1.00,1.40,1.05,0.11,6.80,7.98,5.04,9.74,72.00,73.00,70.50,74.50,1.00,1.00,1.00,1.00,0,0,1,정상
122564,2024-11-21 02:31:12,VC6M6068,37.192677,126.778407,100.0,8.3,72.0,1.2,1.2,0.0,-1.2,0.0,0.0,-89.7,0.3,2024-11-21 02:00:00,571,화성,37.19525000,126.82094000,3.787075,0.00,0.00,6.50,89.50,6.40,8.30,7.36,0.42,69.00,75.00,72.79,1.23,1.00,1.40,1.05,0.11,6.80,7.98,5.04,9.74,72.00,73.00,70.50,74.50,1.00,1.00,1.00,1.00,0,0,1,정상
111985,2024-11-21 02:41:12,VC6M6068,37.313690,126.753128,100.0,8.2,72.0,1.2,1.2,0.0,-1.2,0.0,-1.5,-88.5,0.3,2024-11-21 02:00:00,435,고잔,37.32713000,126.83397000,7.319178,0.00,0.00,7.50,-99.20,6.40,8.30,7.36,0.42,69.00,75.00,72.79,1.23,1.00,1.40,1.05,0.11,6.80,7.98,5.04,9.74,72.00,73.00,70.50,74.50,1.00,1.00,1.00,1.00,0,0,1,정상
125661,2024-11-21 02:51:12,VC6M6068,37.414622,126.744533,100.0,8.2,72.0,1.2,1.2,0.0,-1.2,0.0,-1.7,-87.9,1.2,2024-11-21 02:00:00,367,신현동,37.40617000,126.78457000,3.666541,-99.50,-99.90,6.00,91.50,6.40,8.30,7.36,0.42,69.00,75.00,72.79,1.23,1.00,1.40,1.05,0.11,6.80,7.98,5.04,9.74,72.00,73.00,70.50,74.50,1.00,1.00,1.00,1.00,0,0,1,정상
133174,2024-11-21 03:01:12,VC6M6068,37.446711,126.637258,100.0,8.3,72.0,1.1,1.1,0.0,-1.1,0.0,0.6,-88.5,1.4,2024-11-21 03:00:00,112,인천,37.47772000,126.62490000,3.611087,1.40,1.40,7.40,86.20,6.40,8.30,7.36,0.42,69.00,75.00,72.79,1.23,1.00,1.40,1.05,0.11,6.80,7.98,5.04,9.74,72.00,73.00,70.50,74.50,1.00,1.00,1.00,1.00,0,0,1,정상
122392,2024-11-21 03:11:12,VC6M6068,37.440431,126.618753,100.0,8.3,72.0,1.0,1.0,0.0,-1.0,0.0,0.5,-88.6,1.3,2024-11-21 03:00:00,112,인천,37.47772000,126.62490000,4.174181,1.40,1.40,7.40,86.20,6.40,8.30,7.36,0.42,69.00,75.00,72.79,1.23,1.00,1.40,1.05,0.11,6.80,7.98,5.04,9.74,72.00,73.00,70.50,74.50,1.00,1.00,1.00,1.00,0,0,0,정상
